## Seedling - ResNet

In [5]:
import numpy as np 
import pandas as pd 
import os
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import *
from keras.models import load_model
from sklearn.model_selection import StratifiedShuffleSplit

In [2]:
model = load_model('model_resnet50.h5')

In [3]:
sharpTrainImage = np.load(os.path.join(os.getcwd(),'Image224/sharpTrainImage.npy'))
trainLabels = np.load(os.path.join(os.getcwd(),'Image224/trainLabels.npy'))
print("Numbers of sharpTrainImage images", len(sharpTrainImage))
print("Numbers of trainLabels images", len(trainLabels))
print(model)

Numbers of sharpTrainImage images 4750
Numbers of trainLabels images 4750


In [6]:
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.16, random_state=42) # Want a balanced split for all the classes

for train_index, test_index in sss.split(sharpTrainImage, trainLabels):
    print("Using {} for training and {} for validation".format(len(train_index), len(test_index)))
    x_train, x_valid = sharpTrainImage[train_index], sharpTrainImage[test_index]
    y_train, y_valid = trainLabels[train_index], trainLabels[test_index]

Using 3990 for training and 760 for validation


In [7]:
len(x_valid)

760

In [8]:
from keras import optimizers
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import *
from keras.applications.resnet50 import preprocess_input

In [11]:
learning_rate = 0.0001

In [12]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(lr=learning_rate), metrics=['accuracy'])

In [9]:
def label_n2s(x):
    return {
        0: 'Black-grass',
        1: 'Charlock',
        2: 'Cleavers',
        3: 'Common Chickweed',
        4: 'Common wheat', 
        5: 'Fat Hen',
        6: 'Loose Silky-bent',
        7: 'Maize',
        8: 'Scentless Mayweed',
        9: 'Shepherds Purse',
        10: 'Small-flowered Cranesbill',
        11: 'Sugar beet'
    }.get(x, 'None')

In [10]:
datagen = ImageDataGenerator(
                            preprocessing_function = preprocess_input,
                            rotation_range=360.,
                            width_shift_range=0.3,
                            height_shift_range=0.3,
                            zoom_range=0.3,
                            horizontal_flip=True, 
                            vertical_flip=True)

In [11]:
predValidLabel = []
for i in range(len(x_valid)):
    images = datagen.flow(np.expand_dims(x_valid[i],axis=0))
    pred = np.zeros((1,12))
    for j,img in enumerate(images):
        pred += model.predict(img)
        if j > 100:
            break
    label = np.argmax(pred)
    predValidLabel.append(label)
    if (i%100==0):
        print("Peocess:",i)

Peocess: 0
Peocess: 100
Peocess: 200
Peocess: 300
Peocess: 400
Peocess: 500
Peocess: 600
Peocess: 700


In [12]:
predValidLabel = np.asarray(predValidLabel)

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
acc_valid = accuracy_score(y_valid, predValidLabel)
cnf_matrix = confusion_matrix(y_valid, predValidLabel)

In [14]:
acc_valid

0.9763157894736842

In [15]:
abbreviation = ['BG', 'Ch', 'Cl', 'CC', 'CW', 'FH', 'LSB', 'M', 'SM', 'SP', 'SFC', 'SB']

In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = [16, 10]
plt.rcParams['font.size'] = 16
fig, ax = plt.subplots(1)
ax = sns.heatmap(cnf_matrix, ax=ax, cmap=plt.cm.Greens, annot=True)
ax.set_xticklabels(abbreviation)
ax.set_yticklabels(abbreviation)
plt.title('Confusion Matrix')
plt.ylabel('True class')
plt.xlabel('Predicted class')
fig.savefig('Confusion matrix.png', dpi=300)
plt.show();